In [1]:
from itertools import product
from operator import itemgetter

import torch
from matplotlib import pyplot as plt

# noinspection PyUnresolvedReferences
from clipppy.patches import torch_numpy
from clipppy.utils.plotting.nre import multi_posterior
from libplotting import ResultsPlotter
from libsimplesn import SimpleSN


torch.set_default_tensor_type(torch.cuda.FloatTensor)

## Generate

In [ ]:
hdis_nre = {}
for N, datatype, version in product((1_000, 2_000, 5_000, 10_000), ('photoz',), (0,)):
    simplesn = SimpleSN('pantheon-g10', N=N, datatype=datatype, suffix=0, version=version)
    config = simplesn.config('simplesn.yaml', gen=True)

    base = ResultsPlotter.base_logdir / simplesn.basedata_prefix

    for vpath in base.glob('**/version_*'):
        plotter = ResultsPlotter(simplesn, config, version=version)
        plotter.logdir = vpath
        plotter.bounds = {
            key: {'lower': lower, 'upper': upper}
            for key, (lower, upper) in simplesn.zoom_bounds[-2].items()
        }

        vrelpath = str(vpath.relative_to(base))

        print(N, datatype, vrelpath)

        x, y = map(
            torch.Tensor.numpy,
            itemgetter('Om0', 'Ode0')(plotter.nrep.plotters['Om0', 'Ode0'].grid)
        )
        z = plotter.perc(simplesn.data).rename_(None).flatten().numpy()

        hdis_nre[simplesn.N, simplesn.datatype, simplesn.suffix, version, vrelpath] = x, y, z

In [4]:
hdis_nre = {}
for N, datatype in product((10_000, 20_000, 50_000, 100_000), ('mphotoz', 'specz')):
    simplesn = SimpleSN(survey='pantheon-g10', datatype=datatype, N=N, suffix=0)
    config = simplesn.config('simplesn-marginal.yaml', gen=True)

    base = ResultsPlotter.base_logdir / simplesn.basedata_prefix
    for vpath in base.glob('**/version_*'):
        plotter = ResultsPlotter(simplesn, config, None)
        plotter.logdir = vpath

        vrelpath = str(vpath.relative_to(base))

        print(N, datatype, vrelpath)
        for data_version in range(10):
            ssn = plotter.simplesn.clone(version=data_version)

            x, y = map(
                torch.Tensor.numpy,
                itemgetter('Om0', 'Ode0')(plotter.nrep.plotters['Om0', 'Ode0'].grid)
            )
            z = plotter.perc(ssn.data).rename(None).flatten().numpy()

            hdis_nre[ssn.N, ssn.datatype, ssn.suffix, data_version, vrelpath] = x, y, z

10000 mphotoz onlycosmo-fc/step/version_0


/u/k/kkarchev/Projects/snai-tossn/demo/simplesn/libplotting.py:82: PerformanceWarning: indexing past lexsort depth may impact performance.
  for key, val in self.simplesn.hdi_bounds[self.simplesn.datatype, self.simplesn.N].to_dict().items()
/u/k/kkarchev/miniconda3/lib/python3.9/site-packages/torch/_tensor.py:922: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/c10/core/TensorImpl.h:1463.)
  return super(Tensor, self).align_to(names)


10000 mphotoz onlycosmo-fc/step/version_1
10000 specz onlycosmo-fc/step/version_0
10000 specz onlycosmo-fc/step/version_1
10000 specz onlycosmo-fc/step/version_2
10000 specz onlycosmo-fc/step/version_3
20000 mphotoz onlycosmo-fc/step/version_0
20000 specz onlycosmo-fc/step/version_0
50000 mphotoz onlycosmo-fc/step/version_0
50000 specz onlycosmo-fc/step/version_0
100000 mphotoz onlycosmo-fc/step/version_0
100000 mphotoz onlycosmo-fc/step/version_1
100000 specz onlycosmo-fc/step/version_0
100000 specz onlycosmo-fc/step/version_1
100000 specz onlycosmo-fc/step/version_2
100000 specz onlycosmo-fc/step/version_3


### Compare MC

In [2]:
for datatype, nre_versions in (('specz', list(range(4))), ('mphotoz', list(range(2)))):
    for N in (10_000, 100_000):
        simplesn = SimpleSN(survey='pantheon-g10', datatype=datatype, N=N, suffix=0)
        config = simplesn.config('simplesn-marginal.yaml', gen=True)
        hdis_mcmc = simplesn.hdi
        
        for nre_version in nre_versions:
            plotter = ResultsPlotter(simplesn, config, nre_version)
            for data_version in range(10):
                ssn = plotter.simplesn.clone(version=data_version)

        #         fig, axs = plotter.plot_nre_mc(ssn, hdis)
        #         fig.savefig(plotter.plotdir / f'{ssn.data_prefix}-nre_mc.pdf')
        #         plt.close()

                fig = multi_posterior(plotter.nre, plotter.nrep, ssn.data)[plotter.nrepper.groups[0]]
                fig.suptitle(ssn.data_prefix)
                fig.savefig(fname := plotter.plotdir / f'{ssn.data_prefix}-nre.pdf')
                plt.close()
                
                print(fname)

/u/k/kkarchev/Projects/snai-tossn/demo/simplesn/libplotting.py:82: PerformanceWarning: indexing past lexsort depth may impact performance.
  for key, val in self.ssn.hdi_bounds[self.ssn.datatype, self.ssn.N].to_dict().items()
/u/k/kkarchev/miniconda3/lib/python3.9/site-packages/torch/_tensor.py:922: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/c10/core/TensorImpl.h:1463.)
  return super(Tensor, self).align_to(names)


lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-0-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-1-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-2-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-3-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-4-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-5-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-6-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0-specz-7-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_0/plots/pantheon-g10-10000-0

/u/k/kkarchev/Projects/snai-tossn/demo/simplesn/libplotting.py:82: PerformanceWarning: indexing past lexsort depth may impact performance.
  for key, val in self.ssn.hdi_bounds[self.ssn.datatype, self.ssn.N].to_dict().items()


lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-0-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-1-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-2-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-3-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-4-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-5-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-6-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-specz-7-nre.pdf
lightning_logs/pantheon-g10-10000-0-specz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0

/u/k/kkarchev/Projects/snai-tossn/demo/simplesn/libplotting.py:82: PerformanceWarning: indexing past lexsort depth may impact performance.
  for key, val in self.ssn.hdi_bounds[self.ssn.datatype, self.ssn.N].to_dict().items()


...

lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-0-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-1-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-2-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-3-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-4-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-5-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-6-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-10000-0-mphotoz-7-nre.pdf
lightning_logs/pantheon-g10-10000-0-mphotoz/onlycosmo-fc/step/ve

/u/k/kkarchev/Projects/snai-tossn/demo/simplesn/libplotting.py:82: PerformanceWarning: indexing past lexsort depth may impact performance.
  for key, val in self.ssn.hdi_bounds[self.ssn.datatype, self.ssn.N].to_dict().items()


lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-0-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-1-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-2-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-3-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-4-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-5-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-6-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_0/plots/pantheon-g10-100000-0-mphotoz-7-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onl

/u/k/kkarchev/Projects/snai-tossn/demo/simplesn/libplotting.py:82: PerformanceWarning: indexing past lexsort depth may impact performance.
  for key, val in self.ssn.hdi_bounds[self.ssn.datatype, self.ssn.N].to_dict().items()


lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-0-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-1-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-2-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-3-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-4-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-5-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-6-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onlycosmo-fc/step/version_1/plots/pantheon-g10-100000-0-mphotoz-7-nre.pdf
lightning_logs/pantheon-g10-100000-0-mphotoz/onl